In [159]:
import pandas as pd
import numpy as np
import operator
import ast
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import string
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
import spacy
import re

In [3]:
data = pd.read_csv('combined-newsqa-data-v1.csv')
print(data.columns)
print(type(data['is_question_bad'][0]))

# remove Q/A pairs that are invalid or missing
#data = data[(data.is_question_bad=='0.0') & (data.is_answer_absent=='0.0')]
data.head()

Index(['story_id', 'question', 'answer_char_ranges', 'is_answer_absent',
       'is_question_bad', 'validated_answers', 'story_text'],
      dtype='object')
<class 'str'>


,story_id,question,answer_char_ranges,is_answer_absent,is_question_bad,validated_answers,story_text
0,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,What was the amount of children murdered?,294:297|None|None,0.0,0.0,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
1,./cnn/stories/c48228a52f26aca65c31fad273e66164...,Where was one employee killed?,34:60|1610:1618|34:60,0.0,0.0,NaN,(CNN) -- Fighting in the volatile Sudanese reg...
2,./cnn/stories/c65ed85800e4535f4bbbfa2c34d7d963...,who did say South Africa did not issue a visa ...,103:127|114:127|839:853,0.0,0.0,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
3,./cnn/stories/0cf66b646e9b32076513c050edf32a79...,How many years old was the businessman?,538:550|538:550,0.0,0.0,NaN,(CNN) -- England international footballer Ste...
4,./cnn/stories/13012604e3203c18df09289dfedd14cd...,What frightened the families?,690:742|688:791|630:646,0.0,0.0,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."


In [4]:
print(data['is_question_bad'][0])

0.0


In [5]:
# 119,633 Q/A's , 12088 articles
print(len(data))
print(len(data['story_text'].unique()))


119633
12088


In [6]:
first_doc = data['story_text'][0]
first_doc

'NEW DELHI, India (CNN) -- A high court in northern India on Friday acquitted a wealthy businessman facing the death sentence for the killing of a teen in a case dubbed "the house of horrors."\n\n\n\nMoninder Singh Pandher was sentenced to death by a lower court in February.\n\n\n\nThe teen was one of 19 victims -- children and young women -- in one of the most gruesome serial killings in India in recent years.\n\n\n\nThe Allahabad high court has acquitted Moninder Singh Pandher, his lawyer Sikandar B. Kochar told CNN.\n\n\n\nPandher and his domestic employee Surinder Koli were sentenced to death in February by a lower court for the rape and murder of the 14-year-old.\n\n\n\nThe high court upheld Koli\'s death sentence, Kochar said.\n\n\n\nThe two were arrested two years ago after body parts packed in plastic bags were found near their home in Noida, a New Delhi suburb. Their home was later dubbed a "house of horrors" by the Indian media.\n\n\n\nPandher was not named a main suspect by 

In [7]:
first_doc[294:297]

'19 '

In [8]:
second_doc = data['story_text'][2]
second_doc

'Johannesburg (CNN) -- Miffed by a visa delay that led the Dalai Lama to cancel a trip to South Africa, Archbishop Desmond Tutu lashed out at his government Tuesday, saying it had acted worse than apartheid regimes and had forgotten all that the nation stood for.\n\n\n\n\n\n"When we used to apply for passports under the apartheid government, we never knew until the last moment what their decision was," Tutu said at a news conference. "Our government is worse than the apartheid government because at least you were expecting it from the apartheid government.\n\n\n\n\n\n"I have to say that I can\'t believe this. I really can\'t believe this," Tutu said. "You have to wake me up and tell me this is actually happening here."\n\n\n\n\n\nThe Dalai Lama scrapped his planned trip to South Africa this week after the nation failed to issue him a visa in time, his spokesman said.\n\n\n\n\n\nVisa applications for him and his entourage were submitted to the South African High Commission in New Delhi,

In [9]:
print(second_doc[103:127])
print(second_doc[114:127])
print(second_doc[839:853])

Archbishop Desmond Tutu 
Desmond Tutu 
his spokesman 


Answers given by different human reviewers are in the answer_char_ranges column and the validated_answers column. These values are string index ranges within the document that represent the answer. For each row, we need to determine which character range is the best answer to use and extract it. There are a couple ways we could approach this problem, the first is to have a model that takes the question(str), the story(str), spits out an answer(str).

or we could have it spit back out the string indexes themselves. 

In [26]:
# Remove uneeded columns
df = data[['question','validated_answers','story_text']]
df = df.dropna()
df = df.iloc[0:100] # reduce size for development purposes

df.head()

,question,validated_answers,story_text
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort..."
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi..."
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake..."


In [113]:
# loops through the rows and prints the question along with the first answer given
ground_truth = []
for i, row in df.iterrows():
    try:
        answers = ast.literal_eval(row['validated_answers'])
        sorted_ans = sorted(answers.items(), key=operator.itemgetter(1), reverse=True)
        #print(sorted_ans)
        start, end = sorted_ans[0][0].split(':')
        ground_truth.append((int(start), int(end)))
    except ValueError:
        ground_truth.append(np.nan)
        pass
df['ground_truth'] = ground_truth
df = df.dropna()
print(len(df))
df.head()

80


,question,validated_answers,story_text,ground_truth
0,What was the amount of children murdered?,"{""none"": 1, ""294:297"": 2}","NEW DELHI, India (CNN) -- A high court in nort...","(294, 297)"
2,who did say South Africa did not issue a visa ...,"{""839:853"": 1, ""103:127"": 2}",Johannesburg (CNN) -- Miffed by a visa delay t...,"(103, 127)"
4,What frightened the families?,"{""688:791"": 2, ""690:742"": 1}","BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...","(688, 791)"
6,Who is hiring?,"{""301:324"": 2}",CNN affiliates report on where job seekers are...,"(301, 324)"
8,Iran criticizes who?,"{""63:97"": 2}","TEHRAN, Iran (CNN) -- Iran's parliament speake...","(63, 97)"


In [217]:
final = df.drop(columns=['validated_answers'])
final.head()

,question,story_text,ground_truth
0,What was the amount of children murdered?,"NEW DELHI, India (CNN) -- A high court in nort...","(294, 297)"
2,who did say South Africa did not issue a visa ...,Johannesburg (CNN) -- Miffed by a visa delay t...,"(103, 127)"
4,What frightened the families?,"BAGHDAD, Iraq (CNN) -- At least 6,000 Christi...","(688, 791)"
6,Who is hiring?,CNN affiliates report on where job seekers are...,"(301, 324)"
8,Iran criticizes who?,"TEHRAN, Iran (CNN) -- Iran's parliament speake...","(63, 97)"


In [218]:
full_text = ''
m = 0
for col in ['question','story_text']:
    for text in final[col]:
        m = max(m, len(text))
        full_text += text.lower()
    print(m)
    m=0

# get the set of all characters
characters = tuple(set(full_text))
characters[:5]

68
7361


('?', '.', 'r', '4', ',')

In [219]:
# use enumeration to give the characters integer values
int2char = dict(enumerate(characters))

# create the look up dictionary from characters to the assigned integers
char2int = {char: index for index, char in int2char.items()}
print(char2int)


{'?': 0, '.': 1, 'r': 2, '4': 3, ',': 4, '©': 5, 'c': 6, '(': 7, 'ë': 8, '6': 9, '0': 10, 'q': 11, ']': 12, '»': 13, 'x': 14, 'w': 15, 'v': 16, 'h': 17, 'm': 18, 'k': 19, '"': 20, '+': 21, 'i': 22, '/': 23, ' ': 24, 'n': 25, '5': 26, ':': 27, ';': 28, 'ú': 29, '1': 30, '*': 31, '8': 32, 'y': 33, 'b': 34, ')': 35, 'd': 36, '2': 37, '7': 38, 'é': 39, 'l': 40, 'â': 41, '•': 42, '-': 43, '!': 44, '9': 45, 'p': 46, 'g': 47, 't': 48, 's': 49, '&': 50, 'e': 51, 'j': 52, 'ñ': 53, 'z': 54, '_': 55, '\n': 56, '$': 57, 'a': 58, '[': 59, 'u': 60, 'o': 61, "'": 62, 'ã': 63, 'f': 64, '3': 65}


In [220]:
q_max = 68
story_max = 7361

In [221]:
padding_start = False

for i, row in final.iterrows():
    for col, N in zip(['question','story_text'], [q_max, story_max]):
        enc = np.zeros(N, dtype=np.int32)
        row[col] = np.array([char2int[char] for char in row[col].lower()])
        l = min(N, len(row[col]))
        if padding_start:
            enc[:l] = row[col][:l]
        else:
            enc[N-l:] = row[col][:l]
        row[col] = enc
        
final.head()

,question,story_text,ground_truth
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(294, 297)"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(103, 127)"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(688, 791)"
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(301, 324)"
8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(63, 97)"


In [223]:
#final2 = final.reset_index().drop(columns=['index'])
print(final2.index) 
final2.head()

RangeIndex(start=0, stop=80, step=1)


,question,story_text,ground_truth
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(294, 297)"
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(103, 127)"
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(688, 791)"
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(301, 324)"
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","(63, 97)"


## Train/Test Split

In [224]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(final[['question', 'story_text']], 
                                                    final['ground_truth'], 
                                                    test_size=0.3, 
                                                    random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [228]:
print(X_train.index)
X_train.head()

Int64Index([119, 116,  13,   6, 152,  31, 174, 225,  42, 144, 161,  22, 138,
            223, 213,  63, 211,   4, 159,  39, 132,  47,  18,  66, 160, 170,
            183, 118, 105, 220, 224,  94,   2, 146,  11,  35, 139,  15, 226,
            196, 120,  26,  40,  80],
           dtype='int64')


,question,story_text
119,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
116,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
13,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
152,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [229]:
y_train.head()

119    (2989, 3069)
116    (2551, 2572)
13       (221, 232)
6        (301, 324)
152      (327, 386)
Name: ground_truth, dtype: object

In [230]:
class NewsQADataset(Dataset):
    def __init__(self, X, y, N=400, padding_start=False):
        self.y = y
        self.question = X['question']
        self.story_text =X['story_text']
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.question[idx], self.story_text[idx], self.y[idx]

In [231]:
train_ds = NewsQADataset(X_train, y_train)
valid_ds = NewsQADataset(X_valid, y_valid)

In [233]:
train_ds[119]

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 64, 61,  2, 24, 15, 17, 58, 48, 24, 46, 51, 61, 46, 40, 51,
        24, 52, 60, 49, 48, 24, 36, 61, 25, 62, 48, 24, 48,  2, 60, 49, 48,
        24, 48, 17, 51, 22,  2, 24, 22, 25, 49, 48, 22, 25,  6, 48, 49,  0],
       dtype=int32),
 array([ 0,  0,  0, ..., 40, 64,  1], dtype=int32),
 (2989, 3069))

In [236]:
batch_size = 10
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [239]:
next(iter(train_dl))

TypeError: 'DataLoader' object does not support indexing